1.퍼셉트론 개념에 대해 설명하시오 
- 레이어 속에 존재하는 하나의 식이나오는 한개의 신경망

2. 퍼셉트론의 기본 동작 원리 및 수식을 도식화와 함꼐 작성해주세요.
- 피쳐 4개 $x_1,x_2,x_3,x_4$ 에 대하여 $w_1x_1+w_2x_2+w_3x_3+w_4x_4+b$

3. 활성화함수의 역할을 설명하세요
- 레이어 사이의 연결고리

4. 대표적인 활성화 함수에 대해 설명하세요
- 시그모이드 $ -1 <= x <= +1$
- ReLU $0,x$
- leaky ReLU..? $0.01x , x$
- $ p_i / \Sigma p_k $ .......이름이 뭐엿드라.. 소프트맥스

6. 회귀 모델 구현을 간략하게 코드 작성하세요  
피쳐3개

In [46]:
import torch
import torch.nn.functional as F 
import torch.optim as optim
from torchinfo  import summary
from torchmetrics.regression import MeanSquaredError
import pandas as pd
import numpy as np
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader

featureDF=pd.DataFrame(np.arange(30).reshape(-1,3))
targetDF=pd.DataFrame(np.arange(10).reshape(-1,1))

class RegModel(nn.Module):
    def __init__(self,in_in,out_out,n1,n2):
        super().__init__()
        self.in_layer=nn.Linear(in_in,n1)
        self.h_layer=nn.Linear(n1,n2)
        self.out_layer=nn.Linear(n2,out_out)   
    
    def forward(self,x):
        x=self.in_layer(x)
        x=F.relu(x)
        x=self.h_layer(x)
        x=F.relu(x)
        return self.out_layer(x)


class RegDataSet(Dataset):
    def __init__(self,feature,target):
        super().__init__()
        self.feature=feature
        self.target=target

    def __len__(self):
        return self.feature.shape[0]

    def __getitem__(self,index):
        featureTS=torch.FloatTensor(featureDF.iloc[index].values)
        targetTS=torch.FloatTensor(targetDF.iloc[index].values)
        return featureTS,targetTS

in_in=featureDF.shape[1]
out_out=targetDF.shape[1]
EPOCH=5
BATCH_SIZE=5
dataDS=RegDataSet(featureDF,targetDF)
dataDL=DataLoader(dataDS,batch_size=BATCH_SIZE)
model=RegModel(in_in,out_out,2,5)
optimizer=optim.Adam(model.parameters(),lr=0.01)

loss_hist=[]

for epoch in range(EPOCH):
    loss_total=0
    for feature,target in dataDL:
        model.train()
        pre_y=model(feature)

        loss=MeanSquaredError()(pre_y,target)
        loss_total+=loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    loss_hist.append((loss_total.item())/len(dataDL))

print(loss_hist)
       




[85.63208770751953, 70.50347900390625, 58.47321701049805, 48.897850036621094, 42.12773513793945]


7. 이진분류 모델 구현을 간략하게 코드 작성하세요  
피쳐 5개 클래스 2개 층4개

In [51]:
import torch
import torch.nn.functional as F 
import torch.optim as optim
from torchinfo  import summary
from torchmetrics.classification import BinaryF1Score
import pandas as pd
import numpy as np
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader

featureDF=pd.DataFrame(np.arange(30).reshape(-1,5))
targetDF=pd.DataFrame(np.random.randint(0,2,6))

class BiClassModel(nn.Module):
    def __init__(self,in_in,out_out,n1,n2,n3):
        super().__init__()
        self.in_layer=nn.Linear(in_in,n1)
        self.h1_layer=nn.Linear(n1,n2)
        self.h2_layer=nn.Linear(n1,n3)
        self.out_layer=nn.Linear(n3,out_out)   
    
    def forward(self,x):
        x=self.in_layer(x)
        x=F.relu(x)
        x=self.h1_layer(x)
        x=F.relu
        x=self.h2_layer(x)
        x=F.relu(x)
        return self.out_layer(x)


class RegDataSet(Dataset):
    def __init__(self,feature,target):
        super().__init__()
        self.feature=feature
        self.target=target

    def __len__(self):
        return self.feature.shape[0]

    def __getitem__(self,index):
        featureTS=torch.FloatTensor(featureDF.iloc[index].values)
        targetTS=torch.FloatTensor(targetDF.iloc[index].values)
        return featureTS,targetTS

in_in=featureDF.shape[1]
out_out=targetDF.shape[1]
EPOCH=5
BATCH_SIZE=5
dataDS=RegDataSet(featureDF,targetDF)
dataDL=DataLoader(dataDS,batch_size=BATCH_SIZE)
model=BiClassModel(in_in,out_out,2,3,5)
optimizer=optim.Adam(model.parameters(),lr=0.01)

loss_hist=[]

for epoch in range(EPOCH):
    loss_total=0
    for feature,target in dataDL:
        model.train()
        pre_y=model(feature)

        loss=nn.BCELoss()(pre_y,target)
        loss_total+=loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    loss_hist.append((loss_total.item())/len(dataDL))

print(loss_hist)
       

TypeError: linear(): argument 'input' (position 1) must be Tensor, not function

8. 다중분류모델구현을 간략하게 코드 작성하세요 .  
피쳐 5개 클래스 8개 층 3~5 퍼셉트론 동적

In [53]:
import torch
import torch.nn.functional as F 
import torch.optim as optim
from torchinfo  import summary
from torchmetrics.classification import MulticlassF1Score
import pandas as pd
import numpy as np
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader

featureDF=pd.DataFrame(np.arange(30).reshape(-1,5))
targetDF=pd.DataFrame(np.random.randint(0,8,6))

class MultiClassModel(nn.Module):
    def __init__(self,in_in,out_out,n=[]):
        super().__init__()
        self.in_layer=nn.Linear(in_in,n[0])
        self.h_layer=nn.ModuleList()
        for i in range(len(n)-1):
            self.h_layer.append(n[i],n[i+1])
        self.out_layer=nn.Linear(n[-1],out_out)   
    
    def forward(self,x):
        x=self.in_layer(x)
        x=F.relu(x)
        for model in self.h_layer:
            x=F.relu(model(x))
        x=F.relu(x)
        return self.out_layer(x)


class RegDataSet(Dataset):
    def __init__(self,feature,target):
        super().__init__()
        self.feature=feature
        self.target=target

    def __len__(self):
        return self.feature.shape[0]

    def __getitem__(self,index):
        featureTS=torch.FloatTensor(featureDF.iloc[index].values)
        targetTS=torch.FloatTensor(targetDF.iloc[index].values)
        return featureTS,targetTS

in_in=featureDF.shape[1]
out_out=targetDF.shape[1]
EPOCH=5
BATCH_SIZE=5
dataDS=RegDataSet(featureDF,targetDF)
dataDL=DataLoader(dataDS,batch_size=BATCH_SIZE)
model=MultiClassModel(in_in,out_out,n=[5,4,9,5])
optimizer=optim.Adam(model.parameters(),lr=0.01)

loss_hist=[]

for epoch in range(EPOCH):
    loss_total=0
    for feature,target in dataDL:
        model.train()
        pre_y=model(feature)

        loss=nn.BCELoss()(pre_y,target)
        loss_total+=loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    loss_hist.append((loss_total.item())/len(dataDL))

print(loss_hist)

TypeError: append() takes 2 positional arguments but 3 were given

9. 기울기 소실 개념 및 해결방법을 설명하세요.
- 활성화함수 사용시 기울기가 0 으로 수렴하게 되는 상황이 발생 ReLU로 해결

10. 파이토치의 모델 동작 모드에 대 해관려 함수도 함께 설명하시오.
- model.train() 
    - 학습모드, 가중치의 변동과 다음스텝진행가능 
- model.eval()
    - 검증모드, 가중치의 변동을 멈추고 검증이나, 테스트를 위해 사용